# Query twitter API

In [15]:
os.path.join(os.path.dirname(os.path.dirname(__file__)),'.env')

NameError: name '__file__' is not defined

In [36]:
import os
import requests
# from dotenv import load_dotenv, find_dotenv
from datetime import datetime
import pandas as pd
from dotenv import load_dotenv, find_dotenv


search_url = "https://api.twitter.com/2/tweets/search/recent"
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields

def get_token():
    env_path = os.path.join(os.path.dirname(os.getcwd()),'.env')
    env_path = find_dotenv() # automatic find 
    load_dotenv(env_path)
    token = os.getenv('TWITTER_BEARER_TOKEN','Key missing in env settings')
    return token

def last_date_db():
    """
    Method to retrieve the last date in the DB
    """
    # Construct a BigQuery client object.

    query = """
        SELECT created_at
        FROM `wagon-bootcamp-802.my_dataset.twitter_table`
        ORDER BY created_at DESC
        LIMIT 1
    """
    df = pd.read_gbq(query, dialect='standard')
    return df.iloc[0]['created_at'].tz_localize(None).isoformat()+'Z'


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    bearer_token = get_token()
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def query_twitter(start_time=None):
    query_params = {'query': '#F1','tweet.fields':'created_at,lang', 'start_time':start_time, 'max_results' : '100'}
    json_response = connect_to_endpoint(search_url, query_params)
    return json_response

def func():
    '''
    Pushing results to GBQ
    '''
    most_recent_dt = last_date_db()
    df = pd.DataFrame(query_twitter(most_recent_dt)['data']).iloc[:-1]
    df = df[df['lang'] == 'en']
    df['created_at'] = pd.to_datetime(df['created_at'])
    table_id = 'wagon-bootcamp-802.my_dataset.twitter_table'
    df.to_gbq(table_id, if_exists='append')
    print('Done')
    return df

In [37]:
func()

/home/jeremymarchand/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2009: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


200


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15087.42it/s]

Done


,lang,created_at,text,id
0,en,2022-05-21 12:04:15+00:00,RT @F1: George Russell is up to P2! 👀\n\nHe sp...,1527983610346844162
1,en,2022-05-21 12:04:15+00:00,RT @F1: Barcelona really is anybody's guess! 🍿...,1527983610053156868
2,en,2022-05-21 12:04:15+00:00,RT @F1: 🏁 END OF FP3 - TOP 10 🏁\n\n1 LEC 📸\n2 ...,1527983609344401408
4,en,2022-05-21 12:04:15+00:00,RT @F1: 🏁 END OF FP3 - TOP 10 🏁\n\n1 LEC 📸\n2 ...,1527983608056586240
7,en,2022-05-21 12:04:14+00:00,But some of the most notable action of that se...,1527983604445356033
...,...,...,...,...
91,en,2022-05-21 12:03:38+00:00,RT @F1: Mick Schumacher's brakes are feeling t...,1527983453135831042
92,en,2022-05-21 12:03:38+00:00,RT @F1: Mick Schumacher's brakes are feeling t...,1527983453022703620
93,en,2022-05-21 12:03:37+00:00,RT @ChrisMedlandF1: Ferrari explains it change...,1527983451235835907
95,en,2022-05-21 12:03:37+00:00,RT @AstonMartinF1: Wrapping up practice in Bar...,1527983449478467585
